In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# Make sure we run on Tensowflow 2:
print(tf.__version__)

# Read and prepare dataset
First, we start with defining helper functions to plot time series and reading data from csv:

In [ ]:
import csv
from datetime import datetime, timedelta
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

def plot_series(time, series, format="-", start=0, end=None, title="Asset price history", legend=[]):
    plt.plot(time[start:end], series[start:end], format)
    plt.title(title)
    plt.xlabel("Time")
    plt.ylabel("Value")
    plt.legend(legend)
    plt.grid(True)

def read_data(filepath='../data/BTC-USD.csv'):
    DATE_FORMAT = "%Y-%m-%d"
    time_step = []
    prices = []
    with open(filepath) as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        next(reader) # skip header
        for row in reader:
            if row[4] != 'null':
                time_step.append(datetime.strptime(row[0], DATE_FORMAT).date())
                prices.append(float(row[4]))
    return np.array(time_step), np.array(prices, dtype="float32")

### START CODE HERE ### (≈ 1 line of code)
time, series = # TODO: call read_data() function with the path to your dataset as an argument
### END CODE HERE ###

assert not np.any(np.isnan(series))
plt.figure(figsize=(10, 6))
plot_series(time, series)
plt.show()

Now that we have the time series, let's split it into a training and validation set, so we can start forecasting:

In [ ]:
split_time = int(len(time) * 0.8)
time_train = time[:split_time]
x_train = series[:split_time]
time_valid = time[split_time:]
x_valid = series[split_time:]

plt.figure(figsize=(10, 6))
plot_series(time_train, x_train, title="Training set")
plt.show()

window_size = 20
batch_size = 32
shuffle_buffer_size = 1000

plt.figure(figsize=(10, 6))
plot_series(time_valid, x_valid, title="Validation set")
plt.show()

# Prepare dataset for training on NN
Having dataset loaded in, we have to split it into smaller windows suited for training.
Windows will be shuffled and grouped in batches for parallel training.

In [ ]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    dataset = tf.data.Dataset.from_tensor_slices(series)
    dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))
    dataset = dataset.shuffle(shuffle_buffer).map(lambda window: (window[:-1], window[-1]))
    dataset = dataset.batch(batch_size).prefetch(1)
    return dataset

# EXERCISE 1: Prepare dataset for training
### START CODE HERE ### (≈ 4 lines of code)
max_train = # TODO: find the max value in `x_train` array
            # Hint: use np.max function() https://numpy.org/doc/stable/reference/generated/numpy.maximum.html
            # Hint: to be completely sure we do not get negative values, use `np.abs(my_var)` to get absolute value of the variable
x_train     # TODO: normalize values in `x_train` by dividing them by `max_train`
max_valid = # TODO: find the max value in `x_valid` array
            # Hint: use np.max function() https://numpy.org/doc/stable/reference/generated/numpy.maximum.html
            # Hint: to be completely sure we do not get negative values, use `np.abs(my_var)` to get absolute value of the variable
x_valid # TODO: normalize values in `x_valid` by dividing them by `max_valid`
### END CODE HERE ###

dataset = windowed_dataset(x_train, window_size, batch_size, shuffle_buffer_size)
print("First dataset entry: " + str(list(dataset)[:1]))
# If implemented correctly, you should see the first dataset entry with batches of windows of 20 items each, similar to this:
### First dataset entry: [(<tf.Tensor: shape=(32, 20), dtype=float32, numpy=
### array([[0.01467832, 0.01509622, 0.01453266, 0.0148947 , 0.01409398,
###         0.01362541, 0.01373496, 0.01158416, 0.00913471, 0.01076267,
###         0.01067306, 0.01021982, 0.01078805, 0.01101998, 0.01083811,
###         0.0116373 , 0.01197113, 0.0119441 , 0.0127118 , 0.01301291],

# Create the model
Now when data is prepared we can start creating a NN model.
We will implement a simple 3-layer model with 100-10-1 neurons in each layer respectively.

In [ ]:
# EXERCISE 2: Create the model
model = tf.keras.models.Sequential([
    ### START CODE HERE ### (≈ 3 lines of code)
    # TODO: create a Dense layer with 100 neurons in it. Provide the `[window_size]` as `input_shape` param and use 'relu' as `activation` param
    # TODO: create a Dense layer with 10 neurons in ut. Set activation to 'relu'
    # TODO: create a Dense layer with 10 neurons in ut. Do not provide the activation function param (use default)
    ### END CODE HERE ###
])
model.summary()
# You should see a similar model summary printed:
### Model: "sequential"
### _________________________________________________________________
### Layer (type)                 Output Shape              Param #
### =================================================================
### dense (Dense)                (None, 100)               2100
### _________________________________________________________________
### dense_1 (Dense)              (None, 10)                1010
### _________________________________________________________________
### dense_2 (Dense)              (None, 1)                 11
### =================================================================
### Total params: 3,121
### Trainable params: 3,121
### Non-trainable params: 0

In [ ]:
# EXERCISE 3: Train the model
### START CODE HERE ### (≈ 2 lines of code)
model.compile(YOUR_CODE_HERE)           # TODO: compile the model. Specify "mse" as `loss` param and use `tf.keras.optimizers.SGD` as `optimizer` param
                                        # set SGD params `lr` to 1e-6 and `momentum` to 0.9
history = model.fit(YOUR_CODE_HERE)     # TODO: fit the model on dataset. Also set `epochs` param to, let's say 100 and set `verbose` to 2
                                        # Hint: depending on your dataset you might want to increase the amount of epochs
                                        # In case of Bitcoin dataset, it took me 400-500 epochs to see some meaningful results
### END CODE HERE ###

plt.plot(history.history['loss'])
plt.title("Training Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.grid(True)

# When implemented, you should see that training is started:
### Epoch 1/100
### 61/61 - 0s - loss: 0.0134
### Epoch 2/100
### 61/61 - 0s - loss: 0.0133
### .........

# After training is done, you should the Loss graph displayed with loss value (hopefully) going down with training epochs.

# Forecasting and errors
Congratulations! You've implemented a simple NN that predicts the price of the asset of your choice!

Now let's gather the forecast for the whole series, plot it and see what MSE/MAE values we have achieved.

Note: gathering the forecast takes some time.

In [ ]:
forecast = []
for time in range(len(series) - window_size):
    series_window = series[time:time + window_size]
    series_window_expanded = series_window[np.newaxis]
    predicted_value = model.predict(series_window_expanded)
    forecast.append(predicted_value)

forecast = forecast[split_time-window_size:]
results = np.array(forecast)[:, 0, 0]

# Get back the absolute results to plot the prices and calculate mean errors
results_abs = results * max_valid
x_valid_abs = x_valid * max_valid

plt.figure(figsize=(10, 6))
plot_series(time_valid, x_valid_abs)
plot_series(time_valid, results_abs, title="Deep Learning prediction", legend=["Actual", "Forecast"])

In [ ]:
print("MSE:")
print(tf.keras.metrics.mean_squared_error(x_valid_abs, results_abs).numpy())
print("MAE:")
print(tf.keras.metrics.mean_absolute_error(x_valid_abs, results_abs).numpy())